In [1]:
import sys
sys.path.append ('./model/model')
sys.path.append ('./model/utils')
from keras.models import load_model
#from option import ModelMGPU
import os
import scipy.io.wavfile as wavfile
import numpy as np
import utils
import tensorflow as tf
import matplotlib
from matplotlib import cm
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline
#parameters
people = 2
num_gpu=1

#path
# exp new
model_path = './saved_AO_models/AOmodel-1p-001-0.24099.h5'

Using TensorFlow backend.
/home/kody.uchiyama/.local/share/virtualenvs/asmr_sound_separation-Q8KNsWuG/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kody.uchiyama/.local/share/virtualenvs/asmr_sound_separation-Q8KNsWuG/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kody.uchiyama/.local/share/virtualenvs/asmr_sound_separation-Q8KNsWuG/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, 

In [2]:
database = './data/AV_model_database/mix/'
face_emb_path = '/home/kody.uchiyama/speech_separation-master/data/video/face1022_emb/'
food_truth_path='/home/kody.uchiyama/asmr_sound_separation/data/AV_model_database/food_single/'
print('Initialing Parameters......')

#loading data
print('Loading data ......')
test_file = []
with open('./data/AV_log/AVdataset_test.txt','r') as f:
    test_file = f.readlines()

Initialing Parameters......
Loading data ......


In [3]:
def get_data_name(line,people=people,database=database,face_emb_path=face_emb_path, food_truth_path=food_truth_path):
    parts = line.split() # get each name of file for one testset
    # mix_17-145_00091.npy crm_17-145_00091_17-145.npy 17-145_face_emb.npy
    mix_str = parts[0]
    name_list = mix_str.replace('.npy','')
    name_list = name_list.replace('mix_','')
    names = name_list.split('_')
    single_idxs = [] # 17-145, 00091
    for i in range(2):
        single_idxs.append(names[i])
    file_path1 = database + mix_str
    file_path2 = face_emb_path + parts[2]
    file_path3 = food_truth_path + single_idxs[0] + '.npy'
    mix = np.load(file_path1)
    face_emb = np.load(file_path2)
    food_truth_feature = np.load(file_path3)
    #face_emb_ex = np.zeros((1,75,1,1792,1))
    #face_emb_ex[1,:,:,:,1] = face_emb
    #for i in range(1):
    #    face_emb_ex[1,:,:,:,i] = face_emb

    return mix,single_idxs,face_emb, food_truth_feature

In [4]:
#result predict
ao_model = load_model(model_path,custom_objects={'tf':tf})

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [5]:
##### PLOT RESULTS
def plot_spectrogram(single_idxs, mix, cRM, flag_string):
    # top, bottom title
    if flag_string == 'mix-crm':
        top_title = 'Input'
        bottom_title = 'Mask'
    else:
        top_title = 'Ground truth'
        bottom_title = 'Result'
    
    plt.subplot(2, 1, 1)
    plt.imshow(mix, aspect='auto')
    plt.ylim(0,256)
    plt.title(top_title)
    plt.ylabel('Frequency bands')
    plt.xlabel('Time')
    plt.colorbar()

    plt.subplot(2, 1, 2)
    plt.imshow(cRM, aspect='auto')
    plt.ylim(0,256)
    plt.title(bottom_title)
    plt.ylabel('Frequency bands')
    plt.xlabel('Time')
    plt.colorbar()
    
    plt.tight_layout()#図の調整
    plt.draw()

    #生成されたスペクトログラムと正解値スペクトログラムを比較表示、保存
    # フォルダの作成
    # make output directory
    if flag_string == 'mix-crm':
        folder = 'predict_AO_mix_crm_fig/'
    else:
        folder = 'predict_AO_food_f_fig/'
    if not os.path.exists(folder):
        os.makedirs(folder)
    plt.savefig(folder + "%s_%s.png"%(single_idxs[0],single_idxs[1]))  # ../graphs/predicted_spectrums/{lr:0.000597}-{ws:0.000759}/1.png
    plt.close()
    

In [7]:
count = 0
if num_gpu<=1:
    for line in test_file:
        mix,single_idxs,face_emb, food_truth_feature = get_data_name(line,people,database,face_emb_path)
        print(mix.shape) # (298, 257, 2)
        print(face_emb.shape) # (75, 1, 1792)
        # モデル入力出力用に型変更
        mix_ex = np.expand_dims(mix,axis=0)
        face_emb_ex = np.expand_dims(face_emb,axis=0)
        face_emb_ex = np.expand_dims(face_emb_ex,axis=4)
        print(mix_ex.shape) #(1, 298, 257, 2)
        print(face_emb_ex.shape) #(1, 75, 1, 1792, 1)
        cRMs = ao_model.predict(mix_ex)
        print(cRMs.shape) # (1, 298, 257, 2, 1)
        cRMs = cRMs[0]
        prefix =''
        for idx in single_idxs:
            prefix +=idx+'_'
        cRM =cRMs[:,:,:,0]
        assert cRM.shape ==(298,257,2)
        # mixスペクトログラムにcRMマスクをかける
        F = utils.fast_icRM(mix,cRM)
        T = utils.fast_istft(F,power=False)
        
        # モデル入力出力用に型変更
        cRM_ = utils.real_imag_shrink(cRM)
        F_ = utils.real_imag_shrink(F)
        mix_ = utils.real_imag_shrink(mix)
        food_truth_feature_ = utils.real_imag_shrink(food_truth_feature)
        
        #マスクも複素数なので視覚化のため
        cRM__ = abs(cRM_)
        
        #振幅スペクトログラムにlog（x）をかける
        F__ = np.log(abs(F_))
        mix__ = np.log(abs(mix_))
        food_truth_feature__ = np.log(abs(food_truth_feature_))
        
        # plot
        plot_spectrogram(single_idxs, mix__.T, cRM__.T, 'mix-crm')
        plot_spectrogram(single_idxs, food_truth_feature__.T, F__.T, 'food-f')
        
        count = count + 1
        if count == 15:
            break


(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
(1, 298, 257, 2, 1)
(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
(1, 298, 257, 2, 1)
(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
(1, 298, 257, 2, 1)
(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
(1, 298, 257, 2, 1)
(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
(1, 298, 257, 2, 1)
(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
(1, 298, 257, 2, 1)
(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
(1, 298, 257, 2, 1)
(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
(1, 298, 257, 2, 1)
(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
(1, 298, 257, 2, 1)
(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
(1, 298, 257, 2, 1)
(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
(1, 298, 257, 2, 1)
(298, 257, 2)
(75, 1, 1792)
(1, 298, 257, 2)
(1, 75, 1, 1792, 1)
